<a href="https://colab.research.google.com/github/PorterMathews/VectorDB_Lab/blob/main/embed/VectorDB_Lab_CS452_(starter).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Download datasets from kaggle

import json
import os

if not os.path.exists("lex-fridman-text-embedding-3-large-128.zip"):
  kaggle_json = {"username": "michaeltreynolds","key": "149701be742f30a8a0526762c61beea0"}
  kaggle_dir = os.path.join(os.path.expanduser("~"), ".kaggle")
  os.makedirs(kaggle_dir, exist_ok=True)
  kaggle_config_path = os.path.join(kaggle_dir, "kaggle.json")
  with open(kaggle_config_path, 'w') as f:
    json.dump(kaggle_json, f)

  !kaggle datasets download -d michaeltreynolds/lex-fridman-text-embedding-3-large-128


In [ ]:
# Unzip kaggle data

!unzip lex-fridman-text-embedding-3-large-128.zip
!unzip lex-fridman-text-embedding-3-large-128/*.zip


In [ ]:
# Use specific libraries
!pip install datasets==2.20.0 psycopg2==2.9.9 pgcopy==1.6.0
import psycopg2

In [76]:
# Get your own trial account at timescaledb and paste your own connection string

#TODO
CONNECTION = "REMOVED"

In [ ]:
# Use this if you want to start over on your postgres table!

DROP_TABLE = "DROP TABLE IF EXISTS podcast, segment"
with psycopg2.connect(CONNECTION) as conn:
    cursor = conn.cursor()
    cursor.execute(DROP_TABLE)
    conn.commit() # Commit the changes


In [ ]:
# Useful function that takes a pd.DataFrame and copies it directly into a table.

import pandas as pd
import io
import psycopg2

from typing import List

def fast_pg_insert(df: pd.DataFrame, connection: str, table_name: str, columns: List[str]) -> None:
    """
        Inserts data from a pandas DataFrame into a PostgreSQL table using the COPY command for fast insertion.

        Parameters:
        df (pd.DataFrame): The DataFrame containing the data to be inserted.
        connection (str): The connection string to the PostgreSQL database.
        table_name (str): The name of the target table in the PostgreSQL database.
        columns (List[str]): A list of column names in the target table that correspond to the DataFrame columns.

        Returns:
        None
    """
    conn = psycopg2.connect(connection)
    _buffer = io.StringIO()
    df.to_csv(_buffer, sep=";", index=False, header=False)
    _buffer.seek(0)
    with conn.cursor() as c:
        c.copy_from(
            file=_buffer,
            table=table_name,
            sep=";",
            columns=columns,
            null=''
        )
    conn.commit()
    conn.close()

Database Schema
We will create a database with two tables: podcast and segment:

**podcast**

- PK: id
 - The unique podcast id found in the huggingface data (i,e., TRdL6ZzWBS0  is the ID for Jed Buchwald: Isaac Newton and the Philosophy of Science | Lex Fridman Podcast #214)
- title
 - The title of podcast (ie., Jed Buchwald: Isaac Newton and the Philosophy of Science | Lex Fridman Podcast #214)

**segment**

- PK: id
 - the unique identifier for the podcast segment. This was created by concatenating the podcast idx and the segment index together (ie., "0;1") is the 0th podcast and the 1st segment
This is present in the as the "custom_id" field in the `embedding.jsonl` and batch_request.jsonl files
- start_time
 - The start timestamp of the segment
- end_time
 - The end timestamp of the segment
- content
 - The raw text transcription of the podcast
- embedding
 - the 128 dimensional vector representation of the text
- FK: podcast_id
 - foreign key to podcast.id

In [ ]:
# Sample document:
# {
#   "custom_id": "89:115",
#   "url": "/v1/embeddings",
#   "method": "POST",
#   "body": {
#     "input": " have been possible without these approaches?",
#     "model": "text-embedding-3-large",
#     "dimensions": 128,
#     "metadata": {
#       "title": "Podcast: Boris Sofman: Waymo, Cozmo, Self-Driving Cars, and the Future of Robotics | Lex Fridman Podcast #241",
#       "podcast_id": "U_AREIyd0Fc",
#       "start_time": 484.52,
#       "stop_time": 487.08
#     }
#   }
# }

# Sample embedding:
# {
#   "id": "batch_req_QZBmHS7FBiVABxcsGiDx2THJ",
#   "custom_id": "89:115",
#   "response": {
#     "status_code": 200,
#     "request_id": "7a55eba082c70aca9e7872d2b694f095",
#     "body": {
#       "object": "list",
#       "data": [
#         {
#           "object": "embedding",
#           "index": 0,
#           "embedding": [
#             0.0035960325,
#             126 more lines....
#             -0.093248844
#           ]
#         }
#       ],
#       "model": "text-embedding-3-large",
#       "usage": {
#         "prompt_tokens": 7,
#         "total_tokens": 7
#       }
#     }
#   },
#   "error": null
# }

In [ ]:
# Create table statements that you'll write
#TODO


# may need to run this to enable vector data type if you didn't select AI in service
# CREATE_EXTENSION = "CREATE EXTENSION vector"

# TODO: Add create table statement
CREATE_PODCAST_TABLE = """
CREATE TABLE IF NOT EXISTS podcasts (
    podcast_id VARCHAR(255) PRIMARY KEY,
    title TEXT,
    last_updated TIMESTAMP WITH TIME ZONE DEFAULT CURRENT_TIMESTAMP
);
"""

CREATE_SEGMENT_TABLE = """
CREATE TABLE IF NOT EXISTS segments (
    segment_id SERIAL PRIMARY KEY,
    custom_id VARCHAR(255) UNIQUE NOT NULL,
    podcast_id VARCHAR(255) REFERENCES podcasts(podcast_id) ON DELETE CASCADE,
    input_text TEXT,
    embedding_model VARCHAR(255),
    dimensions INTEGER,
    start_time FLOAT,
    stop_time FLOAT,
    embedding VECTOR(128),
    response_status_code INTEGER,
    response_request_id VARCHAR(255),
    usage_prompt_tokens INTEGER,
    usage_total_tokens INTEGER,
    embedding_object_type VARCHAR(50),
    embedding_data_object_type VARCHAR(50),
    embedding_data_index INTEGER,
    error_message TEXT,
    created_at TIMESTAMP WITH TIME ZONE DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP WITH TIME ZONE DEFAULT CURRENT_TIMESTAMP
);
"""

conn = psycopg2.connect(CONNECTION)
# TODO: Create tables with psycopg2 (example: https://www.geeksforgeeks.org/executing-sql-query-with-psycopg2-in-python/)


conn.commit()
conn.close()


In [ ]:
## Extract needed data out of JSONL files. This may be the hard part!

# TODO: What data do we need?
# TODO: What data is in the documents jsonl files?
# TODO: What data is in the embedding jsonl files?
# TODO: Get some pandas data frames for our two tables so we can copy the data in!



In [ ]:
#### Optional #####
# In addition to the embedding and document files you might like to load
# the full podcast raw data via the hugging face datasets library

# from datasets import load_dataset
# ds = load_dataset("Whispering-GPT/lex-fridman-podcast")


In [ ]:
# TODO Copy all the "podcast" data into the podcast postgres table!

import pandas as pd
import io
import psycopg2
import json
import os # Import the os module for file system operations
import gc # Import garbage collector interface
from pathlib import Path # Using pathlib for easier path manipulation
from typing import List, Dict, Any, Tuple

# --- Create Tables ---
conn_outer = None # Define conn_outer outside try block for broader scope
try:
    conn_outer = psycopg2.connect(CONNECTION)
    with conn_outer.cursor() as cur:
        # Enable vector extension if not already enabled (run once as superuser or db owner if needed)
        # print("Attempting to create vector extension if it doesn't exist...")
        # cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
        # print("Vector extension checked/created.")

        print("Attempting to create vector extension if it doesn't exist...")
        cur.execute("CREATE EXTENSION IF NOT EXISTS vector;") # <--- THIS LINE
        print("Vector extension checked/created.")

        print("Creating podcasts table...")
        cur.execute(CREATE_PODCAST_TABLE)
        print("Podcasts table created or already exists.")

        print("Creating segments table...")
        cur.execute(CREATE_SEGMENT_TABLE)
        print("Segments table created or already exists.")
    conn_outer.commit()
    print("Tables committed successfully.")
except psycopg2.Error as e:
    print(f"Database error during table creation: {e}")
    if conn_outer:
        conn_outer.rollback()
except Exception as e:
    print(f"A general error occurred: {e}")
finally:
    if conn_outer:
        conn_outer.close()
        print("Database connection closed.")


# --- Data Extraction and Loading ---

# Define BASE paths to the directories containing the files
# IMPORTANT: Replace these with the actual paths to your 'documents' and 'embedding' parent directories
BASE_DOCUMENTS_DIR = Path("documents/documents") # e.g., ./data/documents
BASE_EMBEDDINGS_DIR = Path("embedding/embedding") # e.g., ./data/embedding

def load_jsonl(file_path: Path) -> List[Dict[str, Any]]:
    """Loads a JSONL file from a Path object into a list of dictionaries."""
    data = []
    if not file_path.is_file():
        print(f"Error: File not found at {file_path}")
        return []
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            for i, line in enumerate(f):
                try:
                    data.append(json.loads(line))
                except json.JSONDecodeError as e:
                    print(f"Skipping line {i+1} due to JSONDecodeError in {file_path.name}: {e} - Line: {line.strip()}")
        return data
    except Exception as e:
        print(f"An error occurred while reading {file_path}: {e}")
        return []

# --- vvv THIS IS THE FUNCTION TO REPLACE vvv ---
# Delete the old find_file_pairs function definition (the one matching by name)
# And insert this new definition:

def find_file_pairs(doc_dir: Path, emb_dir: Path) -> List[Tuple[Path, Path]]:
    """
    Finds pairs of document and embedding files by sorting the file lists
    from each directory and pairing them element-wise.
    Assumes both directories have the same number of files and the Nth
    sorted file in docs corresponds to the Nth sorted file in embeddings.
    """
    print("-" * 20)
    print(f"Attempting to find files in Document Dir: {doc_dir.resolve()}")
    if not doc_dir.exists(): print(f"ERROR: Document directory does not exist: {doc_dir}")
    if not os.access(str(doc_dir), os.R_OK): print(f"ERROR: No read permissions for Document directory: {doc_dir}")

    print(f"Attempting to find files in Embedding Dir: {emb_dir.resolve()}")
    if not emb_dir.exists(): print(f"ERROR: Embedding directory does not exist: {emb_dir}")
    if not os.access(str(emb_dir), os.R_OK): print(f"ERROR: No read permissions for Embedding directory: {emb_dir}")
    print("-" * 20)

    # Get all files directly within the specified directories
    doc_files_list = [f for f in doc_dir.glob('*') if f.is_file()]
    emb_files_list = [f for f in emb_dir.glob('*') if f.is_file()]

    # Sort both lists alphabetically by filename
    doc_files_list.sort(key=lambda x: x.name)
    emb_files_list.sort(key=lambda x: x.name)

    print(f"\nFound and sorted {len(doc_files_list)} files in {doc_dir}:")
    if not doc_files_list: print("  <None>")
    for f in doc_files_list: print(f"  - {f.name}")

    print(f"\nFound and sorted {len(emb_files_list)} files in {emb_dir}:")
    if not emb_files_list: print("  <None>")
    for f in emb_files_list: print(f"  - {f.name}")

    pairs = []
    num_docs = len(doc_files_list)
    num_embs = len(emb_files_list)

    if num_docs == 0:
        print("\nERROR: No document files found.")
        return []
    if num_embs == 0:
        print("\nERROR: No embedding files found.")
        return []

    if num_docs != num_embs:
        print(f"\nERROR: Mismatch in file counts! Found {num_docs} document files and {num_embs} embedding files. Cannot pair by order.")
        return [] # Stop processing

    print(f"\nPairing {num_docs} files based on sorted order:")
    for i in range(num_docs):
        doc_path = doc_files_list[i]
        emb_path = emb_files_list[i]
        print(f"  Pair {i+1}: '{doc_path.name}' <--> '{emb_path.name}'")
        pairs.append((doc_path, emb_path))

    print("-" * 20)
    return pairs

# --- ^^^ END OF FUNCTION TO REPLACE ^^^ ---


# --- Main Processing Loop ---

# 1. Find all file pairs
print(f"Searching for file pairs in {BASE_DOCUMENTS_DIR} and {BASE_EMBEDDINGS_DIR}...")
# This line correctly calls the (now replaced) find_file_pairs function
file_pairs = find_file_pairs(BASE_DOCUMENTS_DIR, BASE_EMBEDDINGS_DIR)
total_pairs = len(file_pairs)

if total_pairs == 0:
    print("No document/embedding file pairs found or file counts mismatched. Exiting.") # Updated message
else:
    print(f"Found {total_pairs} file pairs to process.")

    # Loop through each pair
    for i, (doc_file_path, emb_file_path) in enumerate(file_pairs):
        print(f"\n--- Processing Pair {i+1}/{total_pairs}: {doc_file_path.name} ---")

        # 2. Load data for the current pair
        print(f"Loading documents from {doc_file_path.name}...")
        raw_documents_data = load_jsonl(doc_file_path)
        if not raw_documents_data:
            print("No document data loaded for this file. Skipping pair.")
            continue
        # Parse docs into DataFrame
        # ... (rest of parsing logic remains the same) ...
        parsed_docs = []
        for doc in raw_documents_data:
            body = doc.get("body", {})
            metadata = body.get("metadata", {})
            parsed_docs.append({
                "custom_id": doc.get("custom_id"), "doc_input_text": body.get("input"),
                "doc_embedding_model": body.get("model"), "doc_dimensions": body.get("dimensions"),
                "doc_podcast_title": metadata.get("title"), "doc_podcast_id": metadata.get("podcast_id"),
                "doc_start_time": metadata.get("start_time"), "doc_stop_time": metadata.get("stop_time"),
            })
        df_docs = pd.DataFrame(parsed_docs)
        print(f"Loaded {len(df_docs)} documents.")


        print(f"Loading embeddings from {emb_file_path.name}...")
        raw_embeddings_data = load_jsonl(emb_file_path)
        if not raw_embeddings_data:
            print("No embedding data loaded for this file. Skipping pair.")
            del df_docs; gc.collect()
            continue
        # Parse embeddings into DataFrame
        # ... (rest of parsing logic remains the same) ...
        parsed_embeddings = []
        for emb in raw_embeddings_data:
            response = emb.get("response"); error_content = emb.get("error")
            entry = {"custom_id": emb.get("custom_id")}
            if response and isinstance(response.get("body"), dict):
                body = response.get("body", {}); data_list = body.get("data", [])
                data_item = data_list[0] if data_list and isinstance(data_list, list) and len(data_list) > 0 else {}
                usage = body.get("usage", {})
                entry.update({
                    "emb_response_status_code": response.get("status_code"), "emb_response_request_id": response.get("request_id"),
                    "emb_embedding_object_type": body.get("object"), "emb_embedding": data_item.get("embedding"),
                    "emb_embedding_data_object_type": data_item.get("object"), "emb_embedding_data_index": data_item.get("index"),
                    "emb_embedding_model": body.get("model"), "emb_usage_prompt_tokens": usage.get("prompt_tokens"),
                    "emb_usage_total_tokens": usage.get("total_tokens"),
                    "emb_error_message": json.dumps(error_content) if error_content is not None else None,
                })
            else:
                entry.update({
                    "emb_response_status_code": response.get("status_code") if response else None, "emb_response_request_id": None,
                    "emb_embedding_object_type": None, "emb_embedding": None, "emb_embedding_data_object_type": None,
                    "emb_embedding_data_index": None, "emb_embedding_model": None, "emb_usage_prompt_tokens": None,
                    "emb_usage_total_tokens": None,
                    "emb_error_message": json.dumps(error_content) if error_content is not None else "Response or response body missing/invalid",
                })
            parsed_embeddings.append(entry)
        df_embeddings = pd.DataFrame(parsed_embeddings)
        print(f"Loaded {len(df_embeddings)} embeddings.")


        # 3. Prepare Podcast Data (for this pair)
        # ... (logic remains the same) ...
        print("Preparing podcast data for this pair...")
        if not df_docs.empty:
            podcasts_df_chunk = df_docs[
                ["doc_podcast_id", "doc_podcast_title"]
            ].copy()
            podcasts_df_chunk.rename(columns={"doc_podcast_id": "podcast_id", "doc_podcast_title": "title"}, inplace=True)
            podcasts_df_chunk.drop_duplicates(subset=["podcast_id"], inplace=True)
            podcasts_df_chunk.dropna(subset=["podcast_id"], inplace=True)
            print(f"Found {len(podcasts_df_chunk)} unique podcasts in this chunk.")

            # 4. Insert Podcast Data (using INSERT ON CONFLICT)
            # ... (logic remains the same) ...
            if not podcasts_df_chunk.empty:
                conn_insert = None; inserted_count = 0; skipped_count = 0
                try:
                    conn_insert = psycopg2.connect(CONNECTION)
                    with conn_insert.cursor() as cur:
                        sql = "INSERT INTO podcasts (podcast_id, title) VALUES (%s, %s) ON CONFLICT (podcast_id) DO NOTHING;"
                        data_tuples = [tuple(x) for x in podcasts_df_chunk.to_numpy()]
                        cur.executemany(sql, data_tuples)
                        inserted_count = cur.rowcount
                        skipped_count = len(podcasts_df_chunk) - (inserted_count if inserted_count != -1 else 0)
                        conn_insert.commit()
                    print(f"Podcast insert committed. Approx {inserted_count} inserted, {skipped_count} skipped (already existed).")
                except Exception as e:
                    print(f"Error inserting podcast data: {e}")
                    if conn_insert: conn_insert.rollback()
                finally:
                    if conn_insert: conn_insert.close()
            else:
                 print("No unique podcast data to insert for this pair.")
        else:
            print("Skipping podcast preparation/insertion as document data was empty.")


        # 5. Prepare Segment Data (for this pair)
        # ... (logic remains the same) ...
        print("Preparing segment data for this pair...")
        if not df_docs.empty and not df_embeddings.empty:
            df_merged = pd.merge(df_docs, df_embeddings, on="custom_id", how="inner")
            if df_merged.empty:
                 print("No matching segments found between documents and embeddings for this pair.")
            else:
                segments_df_chunk = pd.DataFrame({
                    "custom_id": df_merged["custom_id"], "podcast_id": df_merged["doc_podcast_id"],
                    "input_text": df_merged["doc_input_text"],
                    "embedding_model": df_merged["doc_embedding_model"].fillna(df_merged["emb_embedding_model"]),
                    "dimensions": df_merged["doc_dimensions"], "start_time": df_merged["doc_start_time"],
                    "stop_time": df_merged["doc_stop_time"], "embedding": df_merged["emb_embedding"],
                    "response_status_code": df_merged["emb_response_status_code"],
                    "response_request_id": df_merged["emb_response_request_id"],
                    "usage_prompt_tokens": df_merged["emb_usage_prompt_tokens"],
                    "usage_total_tokens": df_merged["emb_usage_total_tokens"],
                    "embedding_object_type": df_merged["emb_embedding_object_type"],
                    "embedding_data_object_type": df_merged["emb_embedding_data_object_type"],
                    "embedding_data_index": df_merged["emb_embedding_data_index"],
                    "error_message": df_merged["emb_error_message"],
                })
                segments_df_chunk.dropna(subset=["custom_id", "podcast_id"], inplace=True)
                print(f"Prepared {len(segments_df_chunk)} segment records for insertion.")

                # 6. Insert Segment Data (using fast_pg_insert)
                # ... (logic remains the same) ...
                if not segments_df_chunk.empty:
                    segment_table_columns = [
                        "custom_id", "podcast_id", "input_text", "embedding_model", "dimensions",
                        "start_time", "stop_time", "embedding", "response_status_code",
                        "response_request_id", "usage_prompt_tokens", "usage_total_tokens",
                        "embedding_object_type", "embedding_data_object_type", "embedding_data_index",
                        "error_message"
                    ]
                    segments_df_insert_chunk = segments_df_chunk[segment_table_columns]
                    print(f"Inserting {len(segments_df_insert_chunk)} segment records...")
                    try:
                        # Ensure fast_pg_insert function is defined above
                        fast_pg_insert(segments_df_insert_chunk, CONNECTION, "segments", segment_table_columns)
                        print(f"Segment insert successful for pair {i+1}.") # Add success message
                    except Exception as e:
                        print(f"Failed to insert segment data for pair {i+1}: {e}")
                else:
                    print("No segment data to insert for this pair after processing.")
        else:
            print("Skipping segment preparation/insertion as document or embedding data was missing/empty.")


        # 7. Clean up memory
        # ... (logic remains the same) ...
        print("Cleaning up DataFrames for this pair...")
        try:
            del df_docs; del df_embeddings
            if 'df_merged' in locals(): del df_merged
            if 'podcasts_df_chunk' in locals(): del podcasts_df_chunk
            if 'segments_df_chunk' in locals(): del segments_df_chunk
            if 'segments_df_insert_chunk' in locals(): del segments_df_insert_chunk
        except NameError: pass
        gc.collect()
        print("Cleanup complete.")

    print("\n--- All file pairs processed ---")


In [ ]:
# TODO Copy all the "segment" data into the segment postgres table!
# HINT 1: use the recommender.utils.fast_pg_insert function to insert data into the database
# otherwise inserting the 800k documents will take a very, very long time
# HINT 2: if you don't want to use all your memory and crash
# colab, you'll need to either send the data up in chunks
# or write your own function for copying it up. Alternative to chunking maybe start
# with writing it to a CSV and then copy it up?


In [77]:
## This script is used to query the database
import os
import psycopg2


# Write your queries
# Q1) What are the five most similar segments to segment "267:476"
# Input: "that if we were to meet alien life at some point"
# For each result return the podcast name, the segment id, segment raw text,  the start time, stop time, and embedding distance

conn = psycopg2.connect(CONNECTION)
cur = conn.cursor()
cur.execute("""
    WITH target_segment AS (
    SELECT embedding
    FROM segments
    WHERE custom_id = '267:476' -- Your target segment's custom_id
    )

    SELECT s.custom_id, input_text, podcast_id, start_time, stop_time, s.embedding <-> (SELECT embedding FROM target_segment) as distance
    FROM segments s, target_segment
    WHERE s.custom_id != '267:476'
    ORDER BY distance ASC
    LIMIT 5;
""")
for row in cur.fetchall():
  print(row)

conn.commit()
conn.close()

('113:2792', ' encounters, human beings, if we were to meet another alien', 'qLDp-aYnR1Y', 6725.62, 6729.86, 0.6483450674336982)
('268:1019', ' Suppose we did meet an alien from outer space', '5f-JlzBuUUU', 2900.04, 2903.0800000000004, 0.6558106859320757)
('305:3600', ' but if we think of alien civilizations out there', 'EwueqdgIvq4', 9479.960000000001, 9484.04, 0.6595433115268592)
('18:464', ' So I think when we meet alien life from outer space,', 'kD5yc1LQrpQ', 1316.8600000000001, 1319.5800000000002, 0.6662026419636159)
('71:989', ' because if aliens come to us', 'SFxIazwNP_0', 2342.34, 2343.6200000000003, 0.6742942635162208)


In [79]:
# Q2) What are the five most dissimilar segments to segment "267:476"
# Input: "that if we were to meet alien life at some point"
# For each result return the podcast name, the segment id, segment raw text, the start time, stop time, and embedding distance

conn = psycopg2.connect(CONNECTION)
cur = conn.cursor()
cur.execute("""
    WITH target_segment AS (
    SELECT embedding
    FROM segments
    WHERE custom_id = '267:476' -- Your target segment's custom_id
    )

    SELECT s.custom_id, input_text, podcast_id, start_time, stop_time, s.embedding <-> (SELECT embedding FROM target_segment) as distance
    FROM segments s, target_segment
    WHERE s.custom_id != '267:476'
    ORDER BY distance DESC
    LIMIT 5;
""")
for row in cur.fetchall():
  print(row)

conn.commit()
conn.close()

('119:218', ' a 73 Mustang Grande in gold?', 'd2bYwYxqJCM', 519.96, 523.8000000000001, 1.6157687685840119)
('133:2006', ' for 94 car models.', '36_rM7wpN5A', 5818.62, 5820.82, 1.5863359073014982)
('283:1488', ' when I called down to get the sauna.', 'uiNpESmPioQ', 3709.34, 3711.1000000000004, 1.572552805197421)
('241:1436', ' which has all the courses pre-installed.', 'J6XcP4JOHmk', 4068.9, 4071.1400000000003, 1.5663319710412156)
('307:3933', ' and very few are first class and some are budget.', 'rIpUf-Vy2JA', 10648.64, 10650.960000000001, 1.5616341289820461)


In [80]:
# Q3) What are the five most similar segments to segment '48:511'

# Input: "Is it is there something especially interesting and profound to you in terms of our current deep learning neural network, artificial neural network approaches and the whatever we do understand about the biological neural network."
# For each result return the podcast name, the segment id, segment raw text,  the start time, stop time, and embedding distance

conn = psycopg2.connect(CONNECTION)
cur = conn.cursor()
cur.execute("""
    WITH target_segment AS (
    SELECT embedding
    FROM segments
    WHERE custom_id = '48:511' -- Your target segment's custom_id
    )

    SELECT s.custom_id, input_text, podcast_id, start_time, stop_time, s.embedding <-> (SELECT embedding FROM target_segment) as distance
    FROM segments s, target_segment
    WHERE s.custom_id != '48:511'
    ORDER BY distance ASC
    LIMIT 5;
""")
for row in cur.fetchall():
  print(row)

conn.commit()
conn.close()

('155:648', ' Is there something interesting to you or fundamental to you about the circuitry of the brain', 'Ktj050DxG7Q', 3798.48, 3805.84, 0.652299685331962)
('61:3707', ' of what we might discover about neural networks?', 'y3Umo_jd5AA', 8498.02, 8500.1, 0.7121050124628524)
('48:512', " And our brain is there. There's some there's quite a few differences. Are some of them to you either interesting or perhaps profound in terms of in terms of the gap we might want to try to close in trying to create a human level intelligence.", '3t06ajvBtl0', 1846.84, 1865.84, 0.7195603322334674)
('276:2642', ' Have these, I mean, small pockets of beautiful complexity. Does that, do cellular automata, do these kinds of emergence and complex systems give you some intuition or guide your understanding of machine learning systems and neural networks and so on?', 'SGzMElJ11Cc', 8628.16, 8646.16, 0.7357217735737499)
('2:152', ' So is there something like that with physics where so deep learning neural net

In [81]:
# Q4) What are the five most similar segments to segment '51:56'

# Input: "But what about like the fundamental physics of dark energy? Is there any understanding of what the heck it is?"
# For each result return the podcast name, the segment id, segment raw text,  the start time, stop time, and embedding distance

conn = psycopg2.connect(CONNECTION)
cur = conn.cursor()
cur.execute("""
    WITH target_segment AS (
    SELECT embedding
    FROM segments
    WHERE custom_id = '51:56' -- Your target segment's custom_id
    )

    SELECT s.custom_id, input_text, podcast_id, start_time, stop_time, s.embedding <-> (SELECT embedding FROM target_segment) as distance
    FROM segments s, target_segment
    WHERE s.custom_id != '51:56'
    ORDER BY distance ASC
    LIMIT 5;
""")
for row in cur.fetchall():
  print(row)

conn.commit()
conn.close()

('308:144', " I mean, we don't understand dark energy, right?", '_L3gNaAVjQ4', 500.44, 502.6, 0.6681965222094363)
('243:273', " Like, what's up with this dark matter and dark energy stuff?", '6ePR2TWYVkI', 946.22, 950.12, 0.7355511762966292)
('196:685', ' being like, what the hell is dark matter and dark energy?', '85F0FDsPHf8', 2591.72, 2595.9599999999996, 0.7631141596843518)
('51:36', ' Do we have any understanding of what the heck that thing is?', 'WxfA1OSev4c', 216.0, 219.0, 0.7922019445543276)
('122:831', ' That is a big question in physics right now.', 's78hvV3QLUE', 2374.9, 2377.6200000000003, 0.8022704628640559)


In [88]:
# Q5) For each of the following podcast segments, find the five most similar podcast episodes. Hint: You can do this by averaging over the embedding vectors within a podcast episode.

#     a) Segment "267:476"

#     b) Segment '48:511'

#     c) Segment '51:56'

# For each result return the Podcast title and the embedding distance

conn = psycopg2.connect(CONNECTION)
cur = conn.cursor()
cur.execute("""
    WITH target_segment_data AS (
      SELECT embedding AS target_embedding, podcast_id AS source_podcast_id
      FROM segments
      WHERE custom_id = '267:476'
    ), average_podcast_embeddings AS (
      SELECT s.podcast_id, title AS podcast_title, AVG(s.embedding) AS avg_episode_embedding
      FROM segments s
      JOIN podcasts p
      ON s.podcast_id = p.podcast_id
      GROUP BY s.podcast_id, title
    )

    SELECT ape.podcast_id, ape.podcast_title, ape.avg_episode_embedding <-> (SELECT target_embedding FROM target_segment_data) AS distance
    FROM average_podcast_embeddings ape, target_segment_data tsd
    WHERE ape.podcast_id != (SELECT source_podcast_id FROM target_segment_data)
    ORDER BY distance ASC
    LIMIT 5;
""")
for row in cur.fetchall():
  print(row)

print()
cur.execute("""
  WITH target_segment_data AS (
    SELECT embedding AS target_embedding, podcast_id AS source_podcast_id
    FROM segments
    WHERE custom_id = '48:511'
  ), average_podcast_embeddings AS (
    SELECT s.podcast_id, title AS podcast_title, AVG(s.embedding) AS avg_episode_embedding
    FROM segments s
    JOIN podcasts p
    ON s.podcast_id = p.podcast_id
    GROUP BY s.podcast_id, title
  )

  SELECT ape.podcast_id, ape.podcast_title, ape.avg_episode_embedding <-> (SELECT target_embedding FROM target_segment_data) AS distance
  FROM average_podcast_embeddings ape, target_segment_data tsd
  WHERE ape.podcast_id != (SELECT source_podcast_id FROM target_segment_data)
  ORDER BY distance ASC
  LIMIT 5;
""")
for row in cur.fetchall():
  print(row)

print()
cur.execute("""
  WITH target_segment_data AS (
    SELECT embedding AS target_embedding, podcast_id AS source_podcast_id
    FROM segments
    WHERE custom_id = '51:56'
  ), average_podcast_embeddings AS (
    SELECT s.podcast_id, title AS podcast_title, AVG(s.embedding) AS avg_episode_embedding
    FROM segments s
    JOIN podcasts p
    ON s.podcast_id = p.podcast_id
    GROUP BY s.podcast_id, title
  )

  SELECT ape.podcast_id, ape.podcast_title, ape.avg_episode_embedding <-> (SELECT target_embedding FROM target_segment_data) AS distance
  FROM average_podcast_embeddings ape, target_segment_data tsd
  WHERE ape.podcast_id != (SELECT source_podcast_id FROM target_segment_data)
  ORDER BY distance ASC
  LIMIT 5;
""")
for row in cur.fetchall():
  print(row)

conn.commit()
conn.close()

('-tDQ74I3Ovs', 'Podcast: Sara Walker: The Origin of Life on Earth and Alien Worlds | Lex Fridman Podcast #198', 0.7828978136062058)
('50r-5ULcWgY', 'Podcast: Martin Rees: Black Holes, Alien Life, Dark Matter, and the Big Bang | Lex Fridman Podcast #305', 0.7879499391348677)
('Gi8LUnhP5yU', 'Podcast: Max Tegmark: Life 3.0 | Lex Fridman Podcast #1', 0.7886899314049058)
('l-NJrvyRo0c', 'Podcast: Sean Carroll: The Nature of the Universe, Life, and Intelligence | Lex Fridman Podcast #26', 0.7890653704600481)
('rfKiTGj-zeQ', 'Podcast: Nick Bostrom: Simulation and Superintelligence | Lex Fridman Podcast #83', 0.7911210354871258)

('piHkfmeU7Wo', 'Podcast: Christof Koch: Consciousness | Lex Fridman Podcast #2', 0.7537802160985114)
('tg_m_LxxRwM', 'Podcast: Dileep George: Brain-Inspired AI | Lex Fridman Podcast #115', 0.7605152893560989)
('aSyZvBrPAyk', 'Podcast: Tomaso Poggio: Brains, Minds, and Machines | Lex Fridman Podcast #13', 0.7615547981858913)
('smK9dgdTl40', 'Podcast: Elon Musk: Neur

In [90]:
# Q6) For podcast episode id = VeH7qKZr0WI, find the five most similar podcast episodes. Hint: you can do a similar averaging procedure as Q5

# Input Episode: "Balaji Srinivasan: How to Fix Government, Twitter, Science, and the FDA | Lex Fridman Podcast #331"
# For each result return the Podcast title and the embedding distance

conn = psycopg2.connect(CONNECTION)
cur = conn.cursor()
cur.execute("""
WITH average_podcast_embeddings AS (
        SELECT s.podcast_id, title AS podcast_title, AVG(s.embedding) AS avg_episode_embedding
        FROM segments s
        JOIN podcasts p
        ON s.podcast_id = p.podcast_id
        GROUP BY s.podcast_id, title
    ), target_episode_data AS (
        SELECT avg_episode_embedding
        FROM average_podcast_embeddings
        WHERE podcast_id = 'VeH7qKZr0WI'
    )

    SELECT ape.podcast_id, ape.podcast_title, ape.avg_episode_embedding <-> (SELECT avg_episode_embedding FROM target_episode_data) AS distance
    FROM average_podcast_embeddings ape, target_episode_data ted
    WHERE ape.podcast_id != 'VeH7qKZr0WI'
    ORDER BY distance ASC
    LIMIT 5;
""")
for row in cur.fetchall():
  print(row)

conn.commit()
conn.close()

('7Grseeycor4', 'Podcast: Tyler Cowen: Economic Growth & the Fight Against Conformity & Mediocrity | Lex Fridman Podcast #174', 0.11950103776872197)
('ifX_JnBfxTY', 'Podcast: Eric Weinstein: Difficult Conversations, Freedom of Speech, and Physics | Lex Fridman Podcast #163', 0.1257139025632404)
('Pl3x4GINtBQ', 'Podcast: Michael Malice and Yaron Brook: Ayn Rand, Human Nature, and Anarchy | Lex Fridman Podcast #178', 0.12842690324343972)
('1XGiTDWfdpM', 'Podcast: Steve Keen: Marxism, Capitalism, and Economics | Lex Fridman Podcast #303', 0.12916269225753493)
('uykM3NhJbso', 'Podcast: Michael Malice: The White Pill, Freedom, Hope, and Happiness Amidst Chaos | Lex Fridman Podcast #150', 0.13040864953585687)


# Deliverables
You will turn in a ZIP or PDF file containing all your code and a PDF file with the queries and results for questions 1-7.